In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

В данном файле показана предподготовка данных. Так как данные для анализа довольно массивные и могут не прогружаться в оперативную память целиком, можем создать вспомогательные файле. Тем более это должно ускорить взаимодействие с некоторыми столбцами, которые в сыром виде требуют предобработки.

Для начала посмотрим на датасет

In [2]:
pd.read_csv('purchases.csv', nrows=1)

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN


**client_id** – id клиента

**transaction_id** – id покупки

**transaction_datetime** – время покупки

**purchase_sum** – сумма покупки

**trn_sum_from_iss**, **product_quantity** содержат информацию о составе покупки – сумма товара и его количество, соответственно. Однако в силу анонимности товаров, эта информация для анализа не подходит.

**regular_points_received**, **express_points_received**, **regular_points_spent**, **express_points_spent** касаются начисления и расходования баллов лояльности. Пока оставим эти столбцы, возможно, потом они будут нам интересны.

In [5]:
pd.read_csv('clients.csv', nrows=1)

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U


**client_id** – id клиента

**first_issue_date** – дата оформления карты лояльности

**first_redeem_date** – дата первой покупки

**age** – возраст клиента

**gender** – пол клиента

Нашей целью будет — убрать упоминание о времени покупки и оформлении карты лояльности, оставим только дату. Также оставим в стороне информацию о содержании покупки, поскольку через **product_id** проблематично оценить, какой это товар. В силу того, что срез в датасете сделан именно по клиентам, а информация о магазине, выраженная с помощью **store_id** также будет не пригодна для анализа (в некоторых магазинах указано несколько, а порой и одна покупка).

In [9]:
purchases = pd.read_csv('purchases.csv', usecols=['client_id', 'transaction_id', 'transaction_datetime', 'purchase_sum'])
clients = pd.read_csv('clients.csv', usecols=['client_id', 'first_issue_date', 'age', 'gender'])

In [10]:
purchases.head()

,client_id,transaction_id,transaction_datetime,purchase_sum
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,1007.0
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,1007.0
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,1007.0
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,1007.0
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,1007.0


In [11]:
clients.head()

,client_id,first_issue_date
0,000012768d,2017-08-05 15:40:48
1,000036f903,2017-04-10 13:54:23
2,000048b7a6,2018-12-15 13:33:11
3,000073194a,2017-05-23 12:56:14
4,00007c7133,2017-05-22 16:17:08


In [12]:
# Оставим только дату
purchases.transaction_datetime = pd.to_datetime(purchases.transaction_datetime).dt.date
# Для ускорения процессов переведем суммы покупок в формат int, копейки не сыграют большой роли в результатах
purchases.purchase_sum = purchases.purchase_sum.astype(int)

Пришло время убрать дубликаты из таблицы. Они возникли в силу того, что каждая запись представляет собой отдельную покупку. Поэтому в графе **transaction_id** для каждого товара указана id покупки и **purchase_sum** сумма покупки в целом.

Так как мы знаем, что в колонках **transaction_datetime**, **purchase_sum** для одинаковых транзакций значения одинаковые, можем ускорить процесс и воспользоваться groupby, а затем агрегировать значения по столбцу, выбирая максимальное. Это будет быстрее, чем drop_duplicates. 

In [17]:
purchases = purchases.groupby(['client_id','transaction_id']).agg({'transaction_datetime': 'max', 
                                                                     'purchase_sum': 'max'}).reset_index()

In [20]:
purchases.head()

,client_id,transaction_id,transaction_datetime,purchase_sum
0,000012768d,6a0e96d0bc,2019-03-08,803
1,000012768d,7e3e2e3984,2018-12-01,1007
2,000012768d,b34f23306e,2019-03-14,419
3,000012768d,c1ca85d462,2018-12-16,574
4,000036f903,0a3d640bf4,2018-12-21,700


In [25]:
# Оставим только дату в столбце таблицы клиентов
clients.first_issue_date = pd.to_datetime(clients.first_issue_date).dt.date

In [26]:
clients.head()

,client_id,first_issue_date,age,gender
0,000012768d,2017-08-05,45,U
1,000036f903,2017-04-10,72,F
2,000048b7a6,2018-12-15,68,F
3,000073194a,2017-05-23,60,F
4,00007c7133,2017-05-22,67,U


In [29]:
# Добавим информацию о клиенте в таблицу с покупками, чтобы не повторять объединение таблиц постоянно
purchases = pd.merge(purchases, clients, on='client_id', how='left')

In [31]:
# Столбец transaction_id нам больше не нужен, поскольку каждая запись представляет собой отдельную покупку, дубликатов больше нет
purchases.drop(['transaction_id'], axis=1, inplace=True)

In [62]:
purchases['days_since_card'] = pd.to_timedelta(purchases.transaction_datetime - purchases.first_issue_date).dt.days
purchases['weeks_since_card'] = purchases.days_since_card // 7
# Важно помнить, что в этом столбце значение 0 показывает, что клиент пользуется картой менее недели

Итак, мы получили таблицу, которая ускорит работу с данными. Поскольку вся работа представляет собой взаимодействие с информацией о покупке, а не о товаре, то, удалив записи о товарах, мы не потеряли ценной информации.

In [68]:
purchases.head()

,client_id,transaction_datetime,purchase_sum,first_issue_date,age,gender,days_since_card,weeks_since_card
0,000012768d,2019-03-08,803,2017-08-05,45,U,580,82
1,000012768d,2018-12-01,1007,2017-08-05,45,U,483,69
2,000012768d,2019-03-14,419,2017-08-05,45,U,586,83
3,000012768d,2018-12-16,574,2017-08-05,45,U,498,71
4,000036f903,2018-12-21,700,2017-04-10,72,F,620,88


Сохраним таблицу в csv.

In [70]:
purchases.to_csv('purchases_processed.csv', index=False)